# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [16]:
import pandas as pd
import numpy as np

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [18]:
loans_url = 'https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true'
loans = pd.read_csv(loans_url, compression='gzip')

c:\Users\maria\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [20]:
# Extraer los nombres de las columnas y sus tipos de datos
column_data = [(col, dtype) for col, dtype in zip(loans.columns, loans.dtypes)]

# Convertir a un DataFrame de pandas
column_types_df = pd.DataFrame(column_data, columns=['column_name', 'type'])

# Mostrar el DataFrame creado
column_types_df

,column_name,type
0,Unnamed: 0,int64
1,id,int64
2,member_id,float64
3,loan_amnt,float64
4,funded_amnt,float64
...,...,...
146,settlement_status,object
147,settlement_date,object
148,settlement_amount,float64
149,settlement_percentage,float64


## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [21]:
datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']

In [22]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [23]:
# COdigo guardar
# Cargar la hoja de cálculo del diccionario de datos de Lending Club en un DataFrame
datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']

# Guardar el DataFrame en un archivo pickle en tu entorno local
pickle_path = 'datos_dict.pkl'  # Ajusta la ruta según sea necesario
datos_dict.to_pickle(pickle_path)

In [24]:
# Codigo para cargar
datos_dict_loaded = pd.read_pickle(pickle_path)

# Mostrar el DataFrame cargado para confirmar
print(datos_dict_loaded.head())

                feature                                        description
0        acc_now_delinq  The number of accounts on which the borrower i...
1  acc_open_past_24mths         Number of trades opened in past 24 months.
2            addr_state  The state provided by the borrower in the loan...
3              all_util              Balance to credit limit on all trades
4            annual_inc  The self-reported annual income provided by th...


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [34]:
# Manejos de tipos 1
# Tu codigo aqui
print(loans.columns) #para ver las columnas
# Nos aseguramos de que todas las fechas estén en formato "datetime"
loans['debt_settlement_flag_date'] = pd.to_datetime(loans['debt_settlement_flag_date'])
loans['settlement_date'] = pd.to_datetime(loans['settlement_date'])
loans['grade'] = loans['grade'].astype('category')

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=148)


In [35]:
# Manejos de tipos 2
# Tu codigo aqui
nuevos_tipos = [(col, dtype) for col, dtype in zip(loans.columns, loans.dtypes)]
column_types_updated = pd.DataFrame(nuevos_tipos, columns=['column_name', 'new_type'])
print(column_types_updated)

               column_name        new_type
0                loan_amnt         float64
1              funded_amnt         float64
2          funded_amnt_inv         float64
3                     term           int32
4                 int_rate         float64
..                     ...             ...
143      settlement_status          object
144        settlement_date  datetime64[ns]
145      settlement_amount         float64
146  settlement_percentage         float64
147        settlement_term         float64

[148 rows x 2 columns]


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

In [47]:

# Imprimir las primeras 5 filas del DataFrame
print("Primeras 5 filas del DataFrame:")
print(loans.head())

# Imprimir información básica sobre el DataFrame
print("\nInformación del DataFrame:")
print(loans.info())

# Imprimir un resumen estadístico de las columnas numéricas
print("\nResumen estadístico:")
print(loans.describe())

# Imprimir los nombres de las columnas
print("\nNombres de las columnas:")
print(loans.columns)

Primeras 5 filas del DataFrame:
   loan_amnt  funded_amnt  funded_amnt_inv  term  int_rate  installment grade  \
0    15000.0      15000.0          15000.0    60     12.39       336.64     C   
1    10400.0      10400.0          10400.0    36      6.99       321.08     A   
2    21425.0      21425.0          21425.0    60     15.59       516.36     D   
3    12800.0      12800.0          12800.0    60     17.14       319.08     D   
4     7650.0       7650.0           7650.0    36     13.66       260.20     C   

  sub_grade                        emp_title emp_length  ...  \
0        C1                       MANAGEMENT  10+ years  ...   
1        A3   Truck Driver Delivery Personel    8 years  ...   
2        D1  Programming Analysis Supervisor    6 years  ...   
3        D4        Senior Sales Professional  10+ years  ...   
4        C3             Technical Specialist   < 1 year  ...   

  hardship_payoff_balance_amount  hardship_last_payment_amount  \
0                    8046.6165

### Imputacion

In [48]:
# Tu codigo aqui
import json

# Asegúrate de que el DataFrame 'loans' esté cargado correctamente
# Por ejemplo: loans = pd.read_csv('tu_archivo.csv')

# Diccionario para registrar las estrategias de imputación
imputacion_dict = {}

for col in loans.columns:
    if loans[col].dtype in ['int64', 'float64']:  # Para datos numéricos
        if loans[col].isnull().all():
            imputacion_dict[col] = {'estrategia': 'todos_nan', 'valor': None}
        else:
            mean_value = loans[col].mean()
            loans[col].fillna(mean_value, inplace=True)
            imputacion_dict[col] = {'estrategia': 'mean', 'valor': mean_value}
    elif loans[col].dtype == 'object':  # Para datos categóricos
        mode_value = loans[col].mode()[0] if not loans[col].mode().empty else 'missing'
        loans[col].fillna(mode_value, inplace=True)
        imputacion_dict[col] = {'estrategia': 'mode', 'valor': mode_value}
    else:
        # Para otros tipos de datos
        imputacion_dict[col] = {'estrategia': 'no_imputada', 'valor': None}

# Guardar el diccionario de imputación en un archivo JSON
json_path = 'estrategias.json'
with open(json_path, 'w') as json_file:
    json.dump(imputacion_dict, json_file, indent=4)


In [49]:
# TUcodigo aqui

# Cargar datos desde un archivo JSON
ruta_json = 'estrategias.json'  # Nombre del archivo JSON que se generó previamente

with open(ruta_json, 'r') as archivo_json:
    datos_cargados = json.load(archivo_json)

# Mostrar los datos cargados
print(datos_cargados)

{'loan_amnt': {'estrategia': 'mean', 'valor': 14886.93}, 'funded_amnt': {'estrategia': 'mean', 'valor': 14886.93}, 'funded_amnt_inv': {'estrategia': 'mean', 'valor': 14883.9105}, 'term': {'estrategia': 'no_imputada', 'valor': None}, 'int_rate': {'estrategia': 'mean', 'valor': 13.278073399995797}, 'installment': {'estrategia': 'mean', 'valor': 437.33182440002366}, 'grade': {'estrategia': 'no_imputada', 'valor': None}, 'sub_grade': {'estrategia': 'mode', 'valor': 'C1'}, 'emp_title': {'estrategia': 'mode', 'valor': 'missing'}, 'emp_length': {'estrategia': 'mode', 'valor': '10+ years'}, 'home_ownership': {'estrategia': 'mode', 'valor': 'MORTGAGE'}, 'annual_inc': {'estrategia': 'mean', 'valor': 74689.24258249998}, 'verification_status': {'estrategia': 'mode', 'valor': 'Source Verified'}, 'issue_d': {'estrategia': 'mode', 'valor': 'Oct-2014'}, 'loan_status': {'estrategia': 'mode', 'valor': 'Fully Paid'}, 'pymnt_plan': {'estrategia': 'mode', 'valor': 'n'}, 'desc': {'estrategia': 'mode', 'valo

### Codigo para salvar y cargar JSONs